In [1]:
import pandas as pd
import numpy as np

In [2]:
t_data = pd.read_table('./datasets/ck_data.txt', header=None, names=['y', 'X'])
t_data

,y,X
0,0,노래가 너무 적음
1,0,"돌겠네 진짜. 황숙아, 어크 공장 그만 돌려라. 죽는다."
2,1,막노동 체험판 막노동 하는사람인데 장비를 내가 사야돼 뭐지
3,1,차악!차악!!차악!!! 정말 이래서 왕국을 되찾을 수 있는거야??
4,1,시간 때우기에 좋음.. 도전과제는 50시간이면 다 깰 수 있어요
...,...,...
99995,0,한글화해주면 10개산다
99996,0,개쌉노잼 ㅋㅋ
99997,0,노잼이네요... 30분하고 지웠어요...
99998,1,야생을 사랑하는 사람들을 위한 짧지만 여운이 남는 이야기. 영어는 그리 어렵지 않습니다.


## 파이썬-프로그래밍 기초

data 관리프로그램
- data수집 및 관리
- data의 분석은 인간관점의 분석을 의미

## 데이터 수집 및 시각화

학습을 진행하기 위한 데이터 수집 방법 학습

## 머신러닝 딥러닝 기초

전문가 시스템
- 인간이 정한 규칙을 전달 규칙에 맞는 답을 제시
1. 머신러닝
    - 머신러닝의 모델은 수학적으로 정의되어있다.
    - 순차적인 프로그래밍
    - 데이터 기반의 학습이지만 수학적 규칙이 정해져있는 모델을 차용한다.
2. 딥러닝
    - 데이터 기반의 학습
    - 데이터를 통한 인간이 인지하지 못하는 부분의 데이터 특징을 컴퓨터 스스로 인식해서 학습하는 전반적인 과정
    - 자연어-> 트랜스포머
    - 이미지-> 트랜스포머
    - 추천 시스템

## 기계학습을 통한 분류기 완성

1. 결측치 확인
2. 중복 확인
3. data 분포도
4. 전처리
5. 모델생성(순환)
6. 모델튜닝 (하이퍼파라미터, 쉘종류, 모델종류, 학습반복수, 뉴런 층수, BN, DO)
7. 결과 기반 동작 확인기(최종적인 분류기 완성)

In [3]:
t_data.isna().sum()

y    0
X    0
dtype: int64

In [4]:
t_data['y'].sum()/t_data.shape[0]

0.49996

In [5]:
from nlclassifiers import KorClassifier
from sklearn.model_selection import train_test_split

8개의 각기 다른 모델 생성
- koc_layer2는 레이어층이 두개 쌓여있고, koc_layer1은 레이어층이 한개
- 두 리스트에 있는 모델들은 서로 전처리 방법이 다름. 같은 리스트에 속하는 모델끼리는 전처리과정이 같음.

In [6]:
t_x, tt_x, t_y, tt_y = train_test_split(t_data['X'], t_data['y'], test_size=0.3, shuffle=True, random_state=4, stratify=t_data['y'])

In [7]:
koc_layer2 = [KorClassifier(max_words=4096, pad_len=29, embedding_size=100, btsize = 256,
                    d_out=0.25, neuron_num=40, bidir=True, layer2=True, gru=False, output_dim=2),
             KorClassifier(max_words=4096, pad_len=29, embedding_size=120, btsize = 256,
                    d_out=0.5, neuron_num=80, bidir=False, layer2=True, gru=True, output_dim=2),
             KorClassifier(max_words=4096, pad_len=29, embedding_size=120, btsize = 256,
                    d_out=0.25, neuron_num=40, bidir=False, layer2=True, gru=False, output_dim=2),
             KorClassifier(max_words=4096, pad_len=29, embedding_size=150, btsize = 256,
                    d_out=0.5, neuron_num=100, bidir=True, layer2=True, gru=True, output_dim=2)]

koc_layer1 = [KorClassifier(max_words=2048, pad_len=21, embedding_size=130, btsize = 256,
                    d_out=0.625, neuron_num=128, bidir=True, layer2=False, gru=False, output_dim=2),
             KorClassifier(max_words=2048, pad_len=21, embedding_size=100, btsize = 256,
                    d_out=0.5, neuron_num=64, bidir=False, layer2=False, gru=True, output_dim=2),
             KorClassifier(max_words=2048, pad_len=21, embedding_size=80, btsize = 256,
                    d_out=0.375, neuron_num=32, bidir=False, layer2=False, gru=False, output_dim=2),
             KorClassifier(max_words=2048, pad_len=21, embedding_size=90, btsize = 256,
                    d_out=0.375, neuron_num=50, bidir=True, layer2=False, gru=True, output_dim=2)]

koc_layer1[0].lbe.fit(t_data['y'])
koc_layer2[0].lbe.fit(t_data['y'])
t_dd = pd.DataFrame()
t_dd['X'] = t_x
t_dd['y'] = t_y
t_x1, t_y1 = koc_layer1[0].preprocessing(t_dd)
t_x2, t_y2 = koc_layer2[0].preprocessing(t_dd)
del t_dd
t_dd = pd.DataFrame()
t_dd['X'] = tt_x
t_dd['y'] = tt_y
tt_x1, tt_y1 = koc_layer1[0].preprocessing(t_dd)
tt_x2, tt_y2 = koc_layer2[0].preprocessing(t_dd)
for i in koc_layer1:
    i.lbe = koc_layer1[0].lbe
    i.tk = koc_layer1[0].tk
for i in koc_layer2:
    i.lbe = koc_layer2[0].lbe
    i.tk = koc_layer2[0].tk

100%|███████████████████████████████████████████████████████████████████████████| 29996/29996 [01:19<00:00, 379.38it/s]


In [8]:
for i in koc_layer1:
    i.fit(t_x1, t_y1, pre_prc = False)
for i in koc_layer2:
    i.fit(t_x2, t_y2, pre_prc = False)

Epoch 1/20
216/216 [==============================] - 35s 147ms/step - loss: 0.5364 - acc: 0.7275 - val_loss: 0.4920 - val_acc: 0.7626
Epoch 2/20
216/216 [==============================] - 32s 146ms/step - loss: 0.4724 - acc: 0.7797 - val_loss: 0.4892 - val_acc: 0.7623
Epoch 3/20
216/216 [==============================] - 32s 149ms/step - loss: 0.4610 - acc: 0.7867 - val_loss: 0.4895 - val_acc: 0.7638
Epoch 4/20
216/216 [==============================] - 32s 150ms/step - loss: 0.4458 - acc: 0.7936 - val_loss: 0.4879 - val_acc: 0.7657
Epoch 5/20
216/216 [==============================] - 37s 171ms/step - loss: 0.4332 - acc: 0.8018 - val_loss: 0.4985 - val_acc: 0.7641
Epoch 6/20
216/216 [==============================] - 34s 155ms/step - loss: 0.4185 - acc: 0.8088 - val_loss: 0.4988 - val_acc: 0.7617
Epoch 1/20
216/216 [==============================] - 10s 39ms/step - loss: 0.5376 - acc: 0.7270 - val_loss: 0.4929 - val_acc: 0.7600
Epoch 2/20
216/216 [==============================] - 8s

In [9]:
for i in koc_layer1:
    i.evaluate(tt_x1, tt_y1, pre_prc = False)
for i in koc_layer2:
    i.evaluate(tt_x2, tt_y2, pre_prc = False)

929/929 [==============================] - 15s 16ms/step - loss: 0.4763 - acc: 0.7714


전반적으로 층을 두개 쌓은 모델이 1% 더 좋은 성능을 보인다.

In [10]:
def review_classifier(x, mod):
    res = mod.predict([x])
    if np.argmax(res)==1:
        print(f'{res[0][1]*100:.1f}% 확률로 긍정리뷰입니다.')
    else:
        print(f'{res[0][0]*100:.1f}% 확률로 부정리뷰입니다.')

분석기가 가장 헷갈려 하는 예제 5개

In [22]:
r_ex = ['일단 노잼까진 아닌데 신선하고 좋긴 함',
        '게임 퀄리티 보면 매우 창렬이다',
        '덜 완성된 느낌이라... 아직까지는 구매를 늦추는게 좋을것 같습니다.',
        '한글화도 엉성하고 게임 자체가 답답함',
        '단순히 재미없다, 재미있다 로 따지면 재밌음'
       ]
for i in r_ex:
    print(i)
    review_classifier(i, koc_layer1[1])

일단 노잼까진 아닌데 신선하고 좋긴 함


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 999.36it/s]


66.9% 확률로 부정리뷰입니다.
게임 퀄리티 보면 매우 창렬이다


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 997.93it/s]


76.6% 확률로 긍정리뷰입니다.
덜 완성된 느낌이라... 아직까지는 구매를 늦추는게 좋을것 같습니다.


...

82.2% 확률로 긍정리뷰입니다.
한글화도 엉성하고 게임 자체가 답답함


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 998.41it/s]

94.2% 확률로 부정리뷰입니다.
단순히 재미없다, 재미있다 로 따지면 재밌음



100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 499.50it/s]


69.3% 확률로 긍정리뷰입니다.


In [12]:
def review_classifier_by_vote(x ,models):
    res_l = [0.0, 0.0]
    for i in models:
        res = i.predict([x])
        res_l[0] += res[0][0] / len(models)
        res_l[1] += res[0][1] / len(models)
    return res_l

In [23]:
for i in r_ex:
    print(i)
    res_l = review_classifier_by_vote(i, koc_layer1 + koc_layer2)
    if np.argmax(res_l)==1:
        print(f'{res_l[1]*100:.1f}% 확률로 긍정리뷰입니다.')
    else:
        print(f'{res_l[0]*100:.1f}% 확률로 부정리뷰입니다.')

일단 노잼까진 아닌데 신선하고 좋긴 함


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 498.61it/s]


52.5% 확률로 긍정리뷰입니다.
게임 퀄리티 보면 매우 창렬이다


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 998.64it/s]


52.5% 확률로 긍정리뷰입니다.
덜 완성된 느낌이라... 아직까지는 구매를 늦추는게 좋을것 같습니다.


...

83.6% 확률로 긍정리뷰입니다.
한글화도 엉성하고 게임 자체가 답답함


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 997.93it/s]


92.4% 확률로 부정리뷰입니다.
단순히 재미없다, 재미있다 로 따지면 재밌음


100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1000.79it/s]

82.2% 확률로 긍정리뷰입니다.
